In [810]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wavfile
import librosa
from numpy import genfromtxt

In [811]:
def cepsmooth(sig, smoothing, falloff, slen):
    tlen = int(round(slen*smoothing/2.0))*2
    hlen = int(round(tlen*falloff/2.0))
    hwin = np.hanning(2*hlen + 2)[1:-1]
    hl = hwin[:hlen]
    hr = hwin[hlen:]

    win = np.concatenate((np.ones(tlen//2-hlen), hr, np.zeros(slen-tlen),
                          hl, np.ones(tlen//2-hlen)))
    win = win.reshape(win.shape[0], 1)
    smspec =\
        np.real(np.fft.ifft(np.log(np.abs(np.fft.fft(sig, slen, axis=0))), axis=0))

    smspec = np.exp(np.real(np.fft.fft(smspec*win, axis=0)))

    # fig, ax = plt.subplots()
    # ax.imshow(win)

    return smspec


In [812]:
def buffer(X, n, p=0, opt=None):
    '''Mimic MATLAB routine to generate buffer array

    MATLAB docs here: https://se.mathworks.com/help/signal/ref/buffer.html

    Parameters
    ----------
    x: ndarray
        Signal array
    n: int
        Number of data segments
    p: int
        Number of values to overlap
    opt: str
        Initial condition options. default sets the first `p` values to zero,
        while 'nodelay' begins filling the buffer immediately.

    Returns
    -------
    result : (n,n) ndarray
        Buffer array created from X
    '''
    import numpy as np

    if opt not in [None, 'nodelay']:
        raise ValueError('{} not implemented'.format(opt))

    i = 0
    first_iter = True
    while i < len(X):
        if first_iter:
            if opt == 'nodelay':
                # No zeros at array start
                result = X[:n]
                i = n
            else:
                # Start with `p` zeros
                result = np.hstack([np.zeros(p), X[:n-p]])
                i = n-p
            # Make 2D array and pivot
            result = np.expand_dims(result, axis=0).T
            first_iter = False
            continue

        # Create next column, add `p` results from last col if given
        col = X[i:i+(n-p)]
        if p != 0:
            col = np.hstack([result[:, -1][-p:], col])
        i += n-p

        # Append zeros if last row and not length `n`
        if len(col) < n:
            col = np.hstack([col, np.zeros(n-len(col))])

        # Combine result with next row
        result = np.hstack([result, np.expand_dims(col, axis=0).T])

    return result


In [813]:
def modgdgram(name):
    srate = 44100
    _, s1 = wavfile.read(name)
    s1 = s1.T / 32768.0  # normalize
    fft_pts = 8192
    m = 441
    n = 1365  # 240
    p, l = s1.shape
    s1 = s1.reshape(l * p)
    nbFrame = int(np.floor((l - n) / m) + 1)
    M1 = buffer(s1, n, n-m)
    win1 = np.hamming(n)[:, np.newaxis]
    smspec = np.zeros((fft_pts, nbFrame))
    range_n = np.arange(n)[:, np.newaxis]
    M2 = M1[:, :nbFrame] * range_n
    M = M1[:, :nbFrame] * win1
    for i in range(nbFrame):
        smspec[:, i,] = np.squeeze(cepsmooth(M[:, i, np.newaxis], 0.01, 0.8, fft_pts))

    fft_M1 = np.fft.fft(M, fft_pts, axis=0)
    fft_M2 = np.fft.fft(M2, fft_pts, axis=0)
    gamma = 0.5
    alpha = 0.9
    r = np.zeros((fft_pts, nbFrame), dtype=np.complex128)
    rf = np.zeros((fft_pts, nbFrame))

    r = np.real(fft_M1) * np.real(fft_M2) + np.imag(fft_M1) * np.imag(fft_M2) / (np.abs(smspec) ** (2 * gamma))
    rf = (r / np.abs(r)) * (np.abs(r) ** alpha)

    return rf


In [814]:
filename = ''


In [815]:
# this is the only function that needs to be called to get a modgdgram
r = modgdgram(filename)

In [1]:
# # Comparison with data from matlab
# matlab_data = genfromtxt('', delimiter=',')
# comparison = np.allclose(matlab_data, r, rtol=0, atol=10**-9)
# comparison

In [818]:
## plotting the result
# plt.figure(figsize=(8.34, 6.25))
# aspect_ratio = float(r[0:1500, :].shape[1]) / float(r[0:1500, :].shape[0])
# plt.imshow(r[0:1500, :],
#            origin="lower",
#            cmap='viridis',
#            aspect=aspect_ratio,
#            vmin = 0, vmax = 1024)
# plt.margins(0, 0)
# plt.xticks([])
# plt.yticks([])
# plt.show()